In [65]:
%load_ext autoreload
%autoreload 2

from dataloader import WICO
import tensorflow as tf
from model import Net
from spektral.data import DisjointLoader
from tensorflow import keras
from tqdm import tqdm
import numpy as np

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_accuracy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
def train_step():
    pass

In [97]:
data = WICO("./dataset/WICO/", root_edges=False, time_delay_edges=False)

In [98]:
np.random.default_rng(0)
np.random.shuffle(data)

train_pct = 0.2
val_pct = 0.1
num_epochs = 5

n = len(data)
train_size, val_size = int(n*train_pct), int(val_pct*n)

train_set = data[:train_size]
val_set = data[train_size:(train_size + val_size)]
test_set = data[(train_size + val_size):]

train_loader = DisjointLoader(train_set, batch_size=1, epochs=num_epochs)
val_loader = DisjointLoader(val_set, batch_size=1, epochs=1)
test_loader = DisjointLoader(test_set, batch_size=1, epochs=1)

In [99]:
model = Net(num_classes=3)
loss_fn = CategoricalCrossentropy(from_logits=True)
optimizer = Adam(learning_rate=0.001)

def train_step(inputs, target):

    with tf.GradientTape() as tape:
        predictions, _, _ = model(X[:2])  # drop edge features from inputs
        loss = loss_fn(target, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    acc = tf.reduce_mean(categorical_accuracy(target, predictions))
    return loss, acc

In [100]:
epoch = step = 0
results = []

for batch in tqdm(train_loader):
    step += 1
    loss, acc = train_step(*batch)
    results.append((loss, acc))

    if step == train_loader.steps_per_epoch:
        step = 0
        epoch += 1
        print("Ep. {} - Loss: {}. Acc: {}".format(epoch, *np.mean(results, 0)))
        results = []

698it [04:07,  2.48it/s]

Ep. 1 - Loss: 0.8249131441116333. Acc: 0.6962750554084778


1396it [08:12,  2.60it/s]

Ep. 2 - Loss: 0.8156884908676147. Acc: 0.7020057439804077


2094it [12:26,  2.59it/s]

Ep. 3 - Loss: 0.8137251734733582. Acc: 0.7020057439804077


2792it [16:43,  2.96it/s]

Ep. 4 - Loss: 0.8159133195877075. Acc: 0.7020057439804077


3490it [20:38,  2.82it/s]

Ep. 5 - Loss: 0.8146923780441284. Acc: 0.7020057439804077


In [103]:
results = []

for batch in test_loader:
    inputs, target = batch
    predictions, _, _ = model(inputs[:2], training=False)
    results.append(
        (
            loss_fn(target, predictions),
            tf.reduce_mean(categorical_accuracy(target, predictions)),
        )
    )
print("Done. Test loss: {}. Test acc: {}".format(*np.mean(results, 0)))

Done. Test loss: 0.7919934391975403. Test acc: 0.7187883853912354


In [63]:


out_log = []


train_loss_results = []
train_accuracy_results = []

epoch_loss_avg = tf.keras.metrics.Mean()
epoch_accuracy = tf.keras.metrics.CategoricalAccuracy()

step = 0

for (features, adj, _), y_true in tqdm(train_loader):
    step += 1
    with tf.GradientTape() as tape:
        # model prediction
        out, _, _ = model([features, adj])            
        y_pred = tf.nn.softmax(out, axis=1)
        loss_val = loss_object(y_true=y_true, y_pred=y_pred)
    
    epoch_loss_avg.update_state(loss_val.numpy())

    epoch_accuracy.update_state(y_true, y_pred)
    grads = tape.gradient(loss_val, model.trainable_variables)

    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    if step % train_loader.steps_per_epoch == 0:
        epoch = step // train_loader.steps_per_epoch

        print(f"epoch={epoch}, loss={epoch_loss_avg.result():.3f}, accuracy={epoch_accuracy.result():.3f}")



15it [00:05,  2.84it/s]

epoch=1, loss=1.096, accuracy=0.467


24it [00:08,  2.77it/s]


In [64]:
train_loader.steps_per_epoch

15